In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
REBUILD_DATE = False

if torch.cuda.is_available():
    DEVICE = torch.device('cuda:0')
    print('Running on GPU')
else:
    DEVICE = torch.device('cpu')
    print('Running on CPU')

Running on GPU


In [3]:
class CatsVsDogs():
    IMAGE_SIZE = 50
    CATS = 'PyTorch/Cat'
    DOGS = 'PyTorch/Dog'
    LABELS = {CATS : 0, DOGS : 1}
    TRAINING_DATA = []
    
    CAT_COUNT = 0
    DOG_COUNT = 0
    
    def Generate_Training_Data(self):
        for LABEL in self.LABELS:
            for FILE in tqdm(os.listdir(LABEL)):
                if 'jpg' in FILE:
                    try:
                        PATH = os.path.join(LABEL, FILE)
                        IMAGE = cv2.imread(PATH, cv2.IMREAD_GRAYSCALE)
                        IMAGE = cv2.resize(IMAGE, (self.IMAGE_SIZE, self.IMAGE_SIZE))
                        self.TRAINING_DATA.append([np.array(IMAGE), np.eye(2)[self.LABELS[LABEL]]])
                        
                        if LABEL == self.CATS:
                            self.CAT_COUNT += 1
                        elif LABEL == self.DOGS:
                            self.DOG_COUNT += 1
                            
                    except Exception as e:
                        pass
                    
        
        np.random.shuffle(self.TRAINING_DATA)
        np.save('TRAINING_DATA.npy', self.TRAINING_DATA)
        print('Cat Count :', self.CAT_COUNT)
        print('Dog Count :', self.DOG_COUNT)

In [4]:
if REBUILD_DATE:
    DATASET = CatsVsDogs()
    DATASET.Generate_Training_Data()
    
TRAINING_DATA = np.load('TRAINING_DATA.npy', allow_pickle = True)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.CONV_1 = nn.Conv2d(1, 32, 5)
        self.CONV_2 = nn.Conv2d(32, 64, 5)
        self.CONV_3 = nn.Conv2d(64, 128, 5)
        
        DUMMY_DATA = torch.randn(50, 50).view(-1, 1, 50, 50)
        self.To_Linear = None
        self.Check_Dim(DUMMY_DATA)
        
        self.FC_1 = nn.Linear(self.To_Linear, 512)
        self.FC_2 = nn.Linear(512, 2)
        
    def Check_Dim(self, X):
        X = F.max_pool2d(F.relu(self.CONV_1(X)), (2, 2))
        X = F.max_pool2d(F.relu(self.CONV_2(X)), (2, 2))
        X = F.max_pool2d(F.relu(self.CONV_3(X)), (2, 2))
        
        if self.To_Linear is None:
            self.To_Linear = X[0].shape[0] * X[0].shape[1] * X[0].shape[2]
        
        return X
    
    def forward(self, X):
        X = self.Check_Dim(X)
        X = X.view(-1, self.To_Linear)
        X = F.relu(self.FC_1(X))
        X = self.FC_2(X)
        
        return F.softmax(X, dim = 1)

In [6]:
NEURAL_NET = Net().to(DEVICE)

OPTIMIZER = optim.Adam(NEURAL_NET.parameters(), lr = 0.001)
LOSS_FUNCTION = nn.MSELoss()

X = torch.Tensor([i[0] for i in TRAINING_DATA]).view(-1, 1, 50, 50)
X = X / 255.0
Y = torch.Tensor([i[1] for i in TRAINING_DATA])

VALIDATION_PERCENTAGE = 0.1
VALIDATION_SIZE = int(len(X) * VALIDATION_PERCENTAGE)

TRAIN_X = X[:-VALIDATION_SIZE]
TRAIN_Y = Y[:-VALIDATION_SIZE]

TEST_X = X[-VALIDATION_SIZE:]
TEST_Y = Y[-VALIDATION_SIZE:]

BATCH_SIZE = 100
EPOCHS = 15

In [7]:
def Train(NEURAL_NET):
    for EPOCH in range(EPOCHS):
        for i in tqdm(range(0, len(TRAIN_X), BATCH_SIZE)):
            BATCH_X = TRAIN_X[i : i + BATCH_SIZE]
            BATCH_Y = TRAIN_Y[i : i + BATCH_SIZE]
            
            BATCH_X, BATCH_Y = BATCH_X.to(DEVICE), BATCH_Y.to(DEVICE)
            
            NEURAL_NET.zero_grad()
            
            OUTPUTS = NEURAL_NET(BATCH_X)
            LOSS = LOSS_FUNCTION(OUTPUTS, BATCH_Y)
            LOSS.backward()
            OPTIMIZER.step()
            
        print('EPOCH :', EPOCH, ' LOSS :', LOSS)
        
Train(NEURAL_NET)

  3%|▎         | 12/450 [00:00<00:04, 109.38it/s]

EPOCH : 0  LOSS : tensor(0.3653, device='cuda:0', grad_fn=<MseLossBackward>)


  5%|▍         | 22/450 [00:00<00:03, 108.90it/s]

EPOCH : 1  LOSS : tensor(0.0724, device='cuda:0', grad_fn=<MseLossBackward>)


  2%|▏         | 11/450 [00:00<00:04, 109.22it/s]

...

EPOCH : 12  LOSS : tensor(1.0939e-08, device='cuda:0', grad_fn=<MseLossBackward>)


  2%|▏         | 11/450 [00:00<00:04, 101.79it/s]

EPOCH : 13  LOSS : tensor(2.2318e-08, device='cuda:0', grad_fn=<MseLossBackward>)


100%|██████████| 450/450 [00:04<00:00, 98.17it/s]

EPOCH : 14  LOSS : tensor(1.2054e-06, device='cuda:0', grad_fn=<MseLossBackward>)


In [8]:
TEST_X.to(DEVICE)
TEST_Y.to(DEVICE)

def Test(NEURAL_NET):
    CORRECT = 0
    TOTAL = 0
    
    with torch.no_grad():
        for i in tqdm(range(len(TEST_X))):
            ACTUAL_CLASS = torch.argmax(TEST_Y[i]).to(DEVICE)
            PREDICTED_OUTPUT = NEURAL_NET(TEST_X[i].view(-1, 1, 50, 50).to(DEVICE))[0]
            PREDICTED_CLASS = torch.argmax(PREDICTED_OUTPUT)
            
            if PREDICTED_CLASS == ACTUAL_CLASS:
                CORRECT += 1
                
            TOTAL += 1
            
    print('ACCURACT :', round(CORRECT / TOTAL, 3 ))
    
Test(NEURAL_NET)

100%|██████████| 4989/4989 [00:06<00:00, 722.80it/s]

ACCURACT : 0.926
